In [27]:
import json
import numpy as np
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [28]:
with open('/content/intents.json') as file:
    data = json.load(file)

In [29]:
data

{'intents': [{'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"],
   'tag': 'thanks'},
  {'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"],
   'tag': 'about'},
  {'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana'],
   'tag': 'name'},
  {'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help?',
    'What can 

In [30]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
training_labels = label_encoder.transform(training_labels)

# Vectorizing our text data using Tokenizer

vocabulary_size = 1200
embedding_dimension = 16
max_length = 20
oov_token = "<OOV>"  # Out Of Token

tokenizer = Tokenizer(num_words = vocabulary_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, maxlen = max_length, truncating = 'post')


# Model Training

model = Sequential()
model.add(Embedding(vocabulary_size, embedding_dimension, input_length = max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()


epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs = epochs)

model.save("chat-model")


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 16)            19200     
                                                                 
 global_average_pooling1d_6   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_18 (Dense)            (None, 16)                272       
                                                                 
 dense_19 (Dense)            (None, 16)                272       
                                                                 
 dense_20 (Dense)            (None, 8)                 136       
                                                                 
Total params: 19,880
Trainable params: 19,880
Non-trainable params: 0
__________________________________________________

INFO:tensorflow:Assets written to: chat-model/assets


In [31]:
# To Save Fitted Tokenizer
with open('tokenizer.pickel', 'wb') as tk:
    pickle.dump(tokenizer, tk, protocol = pickle.HIGHEST_PROTOCOL)
    
# To Save the Fitted Label_Encoder
with open('label_encoder.pickle', 'wb') as le:
    pickle.dump(label_encoder, le, protocol = pickle.HIGHEST_PROTOCOL)


# Time to Check Our Model Performance
with open("intents.json") as file:
    data = json.load(file)

def Chat():
    #model = keras.model.load_model('chat-model')

    #with open('tokenizer.pickle', 'rb') as tk:
        #tokenizer = pickle.load(tk)

    #with open('label_encoder.pickle', 'rb') as le:
        #label_encoder = pickle.load(le)

    max_length = 20

    while True:
        print("User: ", end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), maxlen = max_length,
         truncating = 'post'))

        tag = label_encoder.inverse_transform([np.argmax(result)])

        for i in data["intents"]:
            if i["tag"] == tag:
                print("ChatBot: " , np.random.choice(i["responses"]))
            
print("Type Your Query: (type quit to stop)")
Chat()

hey
Type Your Query: (type quit to stop)
User: ChatBot:  Hi
User: what is your name
ChatBot:  You can call me Joana.
User: Can you help?
ChatBot:  Tell me your problem to assist you
User: how to open a new account
ChatBot:  Just go to our web site and follow the guidelines to create a new account
User: there is a complaint about a service
ChatBot:  Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: thank you
ChatBot:  You're most welcome!
User: bye
ChatBot:  Bye! Come back again
User: quit
Type Your Query: (type quit to stop)
User: 